In [30]:
import importlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ensamble_model as em
importlib.reload(em)

# Train Data
fd = '../../data/main_data/train/train.csv'
train = pd.read_csv(fd)

# Test Data
fd = '../../data/main_data/test/test.csv'
test = pd.read_csv(fd)

In [2]:
general = em.general_Regression(train,type='LR')
general.perform_CV()

MSE for train: mean: 49961930100825.71 std: 1766190834328.662
MSE for test:  mean: 52623128336151.76  std: 16157787424203.045

RMSE for train: mean: 7067261.791999064 std: 125462.60070582716
RMSE for test: mean: 7167938.0340295015 std: 1115254.5343844378

R^2 for train: mean: 0.4080075808979379 std: 0.006344560557093116
R^2 for test: mean: 0.3709262457835678 std: 0.07693190732989633

MAE for train: mean: 3607039.4571535764 std: 52779.976526211336
MAE for test: mean: 3660546.333153973 std: 185378.77754387652

MAPE for train: mean: 4.104013257779082 std: 0.05894752588706118
MAPE for test: mean: 4.1451680329698934 std: 0.5716244798784486



In [3]:
g_model = em.G_Pos(train)
g_model.perform_CV()


MSE for train: mean: 61683219833298.81 std: 2219945018031.943
MSE for test:  mean: 62205878727569.86  std: 20011922777712.785

RMSE for train: mean: 7852586.7440355215 std: 141778.91485141055
RMSE for test: mean: 7783601.32705945 std: 1273353.4894004185

R^2 for train: mean: 0.2691621147664784 std: 0.0052409632861336675
R^2 for test: mean: 0.2639280823504907 std: 0.05308575164317294

MAE for train: mean: 3838273.851915125 std: 54560.33622877767
MAE for test: mean: 3851011.914504575 std: 252278.20128958815

MAPE for train: mean: 4.717831190395815 std: 0.07551720201912203
MAPE for test: mean: 4.728688995878598 std: 0.5542047421151353



In [4]:
d_model = em.D_Pos(train)
d_model.perform_CV()

MSE for train: mean: 50544228181063.92 std: 1802962172237.9333
MSE for test:  mean: 53121591192881.555  std: 16483703522198.58

RMSE for train: mean: 7108305.294912612 std: 127373.52700318863
RMSE for test: mean: 7200370.978870483 std: 1129711.892263365

R^2 for train: mean: 0.40111617463962174 std: 0.006332628828309911
R^2 for test: mean: 0.3656441369508611 std: 0.07528484299740598

MAE for train: mean: 3610349.5081242 std: 51834.37294181964
MAE for test: mean: 3659425.1013683877 std: 192222.99592860552

MAPE for train: mean: 4.061381454039268 std: 0.056254552993016876
MAPE for test: mean: 4.094471598801382 std: 0.5632261402184444



In [5]:
m_model = em.M_Pos(train)
m_model.perform_CV()

MSE for train: mean: 50294405634312.64 std: 1784796950948.8962
MSE for test:  mean: 52895234591291.83  std: 16316294433226.521

RMSE for train: mean: 7090728.837536447 std: 126373.20460311424
RMSE for test: mean: 7185767.778406923 std: 1122486.5371578664

R^2 for train: mean: 0.40407152347115005 std: 0.006349294850707546
R^2 for test: mean: 0.36796804976146796 std: 0.07650070079951457

MAE for train: mean: 3603893.397093645 std: 51871.309000422945
MAE for test: mean: 3654779.804799214 std: 186489.74129049366

MAPE for train: mean: 4.0574758598876555 std: 0.05661994647567225
MAPE for test: mean: 4.094595715678373 std: 0.5810236718762408



In [6]:
f_model = em.F_Pos(train)
f_model.perform_CV()

MSE for train: mean: 50544228181063.92 std: 1802962172237.9333
MSE for test:  mean: 53121591192881.555  std: 16483703522198.58

RMSE for train: mean: 7108305.294912612 std: 127373.52700318863
RMSE for test: mean: 7200370.978870483 std: 1129711.892263365

R^2 for train: mean: 0.40111617463962174 std: 0.006332628828309911
R^2 for test: mean: 0.3656441369508611 std: 0.07528484299740598

MAE for train: mean: 3610349.5081242 std: 51834.37294181964
MAE for test: mean: 3659425.1013683877 std: 192222.99592860552

MAPE for train: mean: 4.061381454039268 std: 0.056254552993016876
MAPE for test: mean: 4.094471598801382 std: 0.5632261402184444



In [7]:
ensamble = em.ensamble_model()
ensamble.perform_CV(train)

MSE for train: mean: 47067774220465.875 std: 1608129583158.1545
MSE for test:  mean: 53069589118872.61  std: 15211355486132.438

RMSE for train: mean: 6859583.605203572 std: 117843.04683879399
RMSE for test: mean: 7212619.981221761 std: 1023573.2144567011

R^2 for train: mean: 0.4422764274646175 std: 0.00610570403278057
R^2 for test: mean: 0.3578213076742 std: 0.09857481553917803

MAE for train: mean: 3581415.0837480677 std: 49290.10083694372
MAE for test: mean: 3711861.345793127 std: 190021.43319106396

MAPE for train: mean: 4.004039934289492 std: 0.05615761669443768
MAPE for test: mean: 4.080809144668858 std: 0.5267677823546241



In [36]:
# Lets try some hyperparameter tuning:

hp = em.hyperparameter_tuning(train,n_iter=2,cv=2)
display(hp.best_params, hp.best_score)

{'G': {'model': 'GBR',
  'param': {'max_depth': 6,
   'n_estimators': 50,
   'min_samples_split': 10,
   'min_samples_leaf': 1,
   'bootstrap': False}},
 'D': {'model': 'RIDGE', 'param': {'alpha': 1.5}},
 'M': {'model': 'LR', 'param': {}},
 'F': {'model': 'RFR',
  'param': {'max_depth': 10,
   'n_estimators': 50,
   'max_features': 0.25,
   'min_samples_split': 10,
   'min_samples_leaf': 1,
   'bootstrap': False}}}

np.float64(7174914.10582092)